# 0. Initialization On Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'Colab Notebooks/21_BigCon'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}/datasets'.format(FOLDERNAME))

# 1. Call Library And Setting Working Directory

In [ ]:
import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
import os

In [ ]:
os.chdir('/content/drive/My Drive/21_BigCon/datasets')

# 2. Load Data

## Korean (내국인 유동인구)

In [ ]:
korean_chunk = pd.read_csv('02-1_KOREAN.CSV', encoding='CP949', chunksize=10**6)
korean = pd.concat([chunk for chunk in korean_chunk])
del korean_chunk

#시 구분(city)과 행정동별코드(emd_cd) 삭제
korean = korean.drop(['time','city', 'emd_cd','resd'], axis=1) 

# 추자면 & 우도면 데이터 제거
idx_=korean[(korean['emd_nm']=='추자면') | (korean['emd_nm']=='우도면')].index  
korean=korean.drop(idx_)
del idx_

In [ ]:
# korean 데이터의 일자를 제거
korean['base_date'] = [korean['base_date'].iloc[d][:7] for d in range(len((korean['base_date'])))]
korean['base_date'] = [korean['base_date'].iloc[d].replace('-','') for d in range(len((korean['base_date'])))]

### 성별&연령대 Encoding

In [ ]:
korean_sum=korean[(korean['sex']=='남성') & (korean['age']==0)]
korean_sum=korean_sum.groupby(['base_date','emd_nm']).sum()
korean_sum.drop(['age'],axis=1,inplace=True)
colnames=['resd_pop_cnt_m0', 'work_pop_cnt_m0', 'visit_pop_cnt_m0']
korean_sum.columns=colnames

for i in range(1, 9):
  a=korean[(korean['sex']=='남성') & (korean['age']==i*10)]
  a=a.groupby(['base_date','emd_nm']).sum()
  a.drop(['age'],axis=1,inplace=True)
  colnames=['resd_pop_cnt_m' + str(i*10), 'work_pop_cnt_m' + str(i*10),
            'visit_pop_cnt_m' + str(i*10)]
  a.columns=colnames
  korean_sum=pd.concat([korean_sum,a],axis=1)

for i in range(0, 9):
  a=korean[(korean['sex']=='여성') & (korean['age']==i*10)]
  a=a.groupby(['base_date','emd_nm']).sum()
  a.drop(['age'],axis=1,inplace=True)
  colnames=['resd_pop_cnt_w' + str(i*10),'work_pop_cnt_w' + str(i*10),
            'visit_pop_cnt_w' + str(i*10)]
  a.columns=colnames
  korean_sum=pd.concat([korean_sum,a],axis=1)

del a 

korean_sum=korean_sum.reset_index()

## card_spending (음식관련 카드소비)



*   base_date가 2018-01-01~2021-06-30을 모두 포함
*   emd_nm이 1개의 "알수없음"을 포함한 44개를 모두 포함

    ==> 따라서 먼저 데이터를 불러옴



In [ ]:
card_spending=pd.read_csv('04_CARD_SPENDING.CSV',encoding='CP949')

#시 구분(city)과 행정동별코드(emd_cd)와 음식 관련 업종 코드(mct_cat_cd) 삭제
card_spending=card_spending.drop(['city', 'emd_cd', 'mct_cat_cd'], axis=1) 

# '우도면'과 '추자도' 데이터 제거
idx_=card_spending[(card_spending['emd_nm']=='추자면') | (card_spending['emd_nm']=='우도면')].index
card_spending=card_spending.drop(idx_)
del idx_

# 음식 관련 11개 업종 dummy변수화
card_spending = pd.get_dummies(card_spending, columns = ['mct_cat_nm'])
card_spending.columns = ['base_date', 'emd_nm', 'use_cnt', 'use_amt', '간식', '농축수산물', '마트/슈퍼마켓', '배달', '부페',
                         '식품', '아시아음식', '양식', '주점및주류판매', '패스트푸드', '한식']

# 월별/행정구역별 합계
card_spending['base_date'] = [card_spending['base_date'].iloc[d][:7] for d in range(len((card_spending['base_date'])))]
card_spending['base_date'] = [card_spending['base_date'].iloc[d].replace('-','') for d in range(len((card_spending['base_date'])))]
card_spending = card_spending.groupby(['base_date', 'emd_nm']).sum()
card_spending = card_spending.reset_index()

In [ ]:
card_spending.info()

## food_waste (음식물쓰레기)


*   city 칼럼은 불필요하다고 판단 -> 삭제
*   43개의 행정동별 코드(emd_cd) or 행정동 이름(emd_nm) 중복된다고 판단 -> 행정동별 코드(emb_cd)는 삭제
*   배출거점지역 코드(em_area_cd) : 행정동 아래 여러 배출거점지역 존재
==> 행정동으로 합치면 되므로 배출거점지역 코드(em_area_cd) 또한 삭제


** 음식점 음식물쓰레기 데이터가 포함 안 됨!!







In [ ]:
food_waste = pd.read_csv('01_FOOD_WASTE.CSV', encoding='CP949')

#시 구분(city)와 행정동별코드(emd_cd), 배출거점지역 코드(em_area_cd) 삭제
food_waste = food_waste.drop(['city', 'emd_cd', 'em_area_cd'], axis=1)

# 월별/행정구역별 합계
food_waste['base_date'] = [food_waste['base_date'].iloc[d][:7] for d in range(len((food_waste['base_date'])))]
food_waste['base_date'] = [food_waste['base_date'].iloc[d].replace('-','') for d in range(len((food_waste['base_date'])))]
food_waste=food_waste.groupby(['base_date', 'emd_nm']).sum()
food_waste=food_waste.reset_index()

In [ ]:
food_waste.info()

## long_term_frgn (장기체류 외국인 유동인구)

In [ ]:
long_term_frgn_chunk = pd.read_csv('02-2_LONG_TERM_FRGN.CSV',encoding='CP949',chunksize=10**6)
long_term_frgn = pd.concat([chunk for chunk in long_term_frgn_chunk])
del long_term_frgn_chunk

#시 구분(city)와 시간(time), 행정동별코드(emd_cd), 거주지역행정구역(resd) 삭제
long_term_frgn = long_term_frgn.drop(['city', 'time', 'emd_cd', 'resd'], axis=1)

# '우도면'과 '추자도' 데이터 제거
idx_=long_term_frgn[(long_term_frgn['emd_nm']=='추자면') | (long_term_frgn['emd_nm']=='우도면')].index
long_term_frgn=long_term_frgn.drop(idx_)
del idx_

# 31개 국가 dummy변수화
long_term_frgn = pd.get_dummies(long_term_frgn, columns = ['nationality'])

# 월별/행정구역별 합계
long_term_frgn['base_date']=pd.to_datetime(long_term_frgn.base_date)
long_term_frgn.index=long_term_frgn.base_date
long_term_frgn=long_term_frgn.groupby(['emd_nm']).resample('M').sum()
long_term_frgn=long_term_frgn.reset_index()
long_term_frgn.base_date=[d.strftime("%Y%m%d") for d in long_term_frgn['base_date']]
long_term_frgn.base_date=[d[:6] for d in long_term_frgn['base_date']]

# 겹치는 column 이름 변경 
long_term_frgn.rename(columns={"resd_pop_cnt": "resd_pop_cnt_lf",
                                "visit_pop_cnt": "visit_pop_cnt_lf",
                               "work_pop_cnt": "work_pop_cnt_lf"}, inplace=True)

In [ ]:
long_term_frgn.info()

## short_term_frgn (단기체류 외국인 유동인구)

In [ ]:
short_term_frgn=pd.read_csv('02-3_SHORT_TERM_FRGN.CSV',encoding='CP949')

# 시간대(time)과 시 구분(city)과 행정동별코드(emd_cd) 삭제
short_term_frgn = short_term_frgn.drop(['time', 'city', 'emd_cd'], axis=1)

# '우도면'과 '추자도' 데이터 제거
idx_=short_term_frgn[(short_term_frgn['emd_nm']=='추자면') | (short_term_frgn['emd_nm']=='우도면')].index
short_term_frgn=short_term_frgn.drop(idx_)
del idx_

# 31개 국가 dummy변수화
short_term_frgn = pd.get_dummies(short_term_frgn, columns = ['nationality'])

# 월별/행정구역별 합계
short_term_frgn['base_date']=pd.to_datetime(short_term_frgn.base_date)
short_term_frgn.index=short_term_frgn.base_date
short_term_frgn=short_term_frgn.groupby(['emd_nm']).resample('M').sum()
short_term_frgn=short_term_frgn.reset_index()
short_term_frgn.base_date=[d.strftime("%Y%m%d") for d in short_term_frgn['base_date']]
short_term_frgn.base_date=[d[:6] for d in short_term_frgn['base_date']]

# 겹치는 column 이름 변경 
short_term_frgn.rename(columns={"visit_pop_cnt": "visit_pop_cnt_sf"}, inplace=True)

In [ ]:
short_term_frgn.info()

## resident_pop (거주인구)

In [ ]:
resident_pop=pd.read_csv('03_RESIDENT_POP.CSV',encoding='CP949')

# 시 구분(city)와 행정동별코드(emd_cd) 삭제
resident_pop = resident_pop.drop(['city', 'emd_cd'], axis=1) 

# '우도면'과 '추자도' 데이터 제거
idx_=resident_pop[(resident_pop['emd_nm']=='추자면') | (resident_pop['emd_nm']=='우도면')].index
resident_pop=resident_pop.drop(idx_)
del idx_

# base_date column 생성
base_month_=list(resident_pop['base_month'])
for i in range(len(base_month_)):
  if base_month_[i] <10:
    base_month_[i] = "0" + str(base_month_[i])
  else:
    str(base_month_[i])

resident_pop['base_date'] = resident_pop['base_year'].astype("str") + base_month_

# base_year 과 base_month column 제거
resident_pop = resident_pop.drop(['base_year', 'base_month'], axis=1) 

# 성별 더미변수화
resident_pop = pd.get_dummies(resident_pop, columns=['sex'])

resident_pop_m = resident_pop[resident_pop['sex_남성']==1].drop(['sex_남성', 'sex_여성'], axis=1)
resident_pop_m.columns =['emd_nm', 'resid_reg_pop_m', 'foreign_pop_m', 'total_pop_m', 'base_date']

resident_pop_w = resident_pop[resident_pop['sex_남성']==0].drop(['sex_남성', 'sex_여성'], axis=1)
resident_pop_w.columns =['emd_nm', 'resid_reg_pop_w', 'foreign_pop_w', 'total_pop_w', 'base_date']

resident_pop = pd.merge(resident_pop_m, resident_pop_w)
del resident_pop_m, resident_pop_w

# 월별/행정동별 합계
resident_pop = resident_pop.groupby(['base_date', 'emd_nm']).sum()
resident_pop=resident_pop.reset_index()

In [ ]:
resident_pop.info()

# 3, Merge Dataset

In [ ]:
data=pd.merge(food_waste,card_spending,how='left',on=['base_date','emd_nm'])
data=pd.merge(data,korean_sum,how='left',on=['base_date','emd_nm'])
data=pd.merge(data,long_term_frgn,how='left',on=['base_date','emd_nm'])
data=pd.merge(data,short_term_frgn,how='left',on=['base_date','emd_nm'])
data=pd.merge(data,resident_pop,how='left',on=['base_date','emd_nm'])

In [ ]:
data.info()

In [ ]:
# dataset 저장
data.to_csv('data.csv')